In [1]:
import pandas as pd
import numpy as np
from datasketch import MinHash, MinHashLSH
from time import time
from sklearn.model_selection import train_test_split
import re

# import nltk
# nltk.download("stopwords")
# Part 2: Nearest Neighbor Search with Locality Sensitive Hashing (LSH)

def preprocess_text(text):
    """
    Preprocess the input text by lowercasing and removing punctuation.
    :param text: The input text string.
    :return: Preprocessed text string.
    """
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w]', '', text)  # Remove punctuation
    return text

def text_to_shingles(text, k=3):
    """
    Convert text into shingles (sets of overlapping word sequences of size k).
    :param text: Input string to be converted into shingles.
    :param k: Shingle size (default is 5).
    :return: A set of k-length shingles.
    """
    text = preprocess_text(text)  # Preprocess the text

    shingles = []
    for i in range(0, len(text) - k):
        shingles.append(text[i:i + k])
    return set(shingles)
    # return [word for word in word_list if word not in stopwords.words('english')]


def jaccard_similarity(doc1: set,doc2:set):
    intersection = doc1.intersection(doc2)
    union = doc1.union(doc2)

    return 1 - len(intersection)/len(union)


def create_minhash(shingle_doc, num_perm):
    m = MinHash(num_perm=num_perm)

    for shingle in shingle_doc:
        m.update(shingle.encode('utf8'))
    return m

def nearest_neighbor_search():
    # Load datasets
    print("Loading the training and the test datasets...")
    train_ds_comma=pd.read_csv("train.csv")
    test_ds_comma=pd.read_csv("test_without_labels.csv")


    fraction=0.10 # the fraction of the datasets, used to create smaller, faster to work on datasets

    # Split the training dataset into a smaller one
    print(f"Splitting the training dataset to a smaller one (fraction:{fraction*100}%) ...")
    train_subset, _ = train_test_split(
        train_ds_comma, 
        train_size=fraction, 
        stratify=train_ds_comma['Label'],  # Ensure stratified sampling
        random_state=42 
    )

    train_subset.to_csv('train_subset.csv', index=False)

    # Split the testing dataset into a smaller one
    print(f"Splitting the testing dataset to a smaller one (fraction:{fraction*100}%) ...")
    test_subset=test_ds_comma.sample(frac=fraction,random_state=42)
    test_subset.to_csv('test_subset.csv',index=False)

    # the two sets that we are going to work on
    train_subset=pd.read_csv('train_subset.csv')
    test_subset=pd.read_csv('test_subset.csv')  

    train_df = train_subset
    test_df = test_subset

    # Example dataset
    train_docs = train_df['Content'] + train_df['Title']
    test_docs = test_df['Content'] + test_df['Title']

    # Preprocessing of the text data
    print("Performing preprocessing and shingling of the data...")

    k_shingle=3

    print(f"Shingling the training set... (k={k_shingle})")
    list_of_train_shingle=[]
    for tr_doc in train_docs:
        sh_doc=text_to_shingles(tr_doc,k=k_shingle)
        list_of_train_shingle.append(sh_doc)

    print(f"Shingling the test set... (k={k_shingle})")

    list_of_test_shingle=[]
    for te_doc in test_docs:
        sh_doc=text_to_shingles(te_doc,k=k_shingle)
        list_of_test_shingle.append(sh_doc)

    
    # Compute K=7 nearest neighbors for each test document
    K = 7
    nearest_neighbors = []
    
    # Brute-force K-NN
    print("Performing Brute-Force K-NN...")
    start_time = time() # we start the clock
    
    for teindex in range(len(list_of_test_shingle)):
        similarities = []
        
        for traindex in range(len(list_of_train_shingle)):
            sim = jaccard_similarity(list_of_test_shingle[teindex], list_of_train_shingle[traindex])
            similarities.append((traindex, sim)) # we keep indexes of train sets
        
        sorted_sim=sorted(similarities,key=lambda x : x[1])
        top_k_neighbors = [idx for idx, sim in sorted_sim[:K]]
        
        nearest_neighbors.append(top_k_neighbors)
    
    brute_force_time = time() - start_time # we stop the clock


    print(f"Brute-Force K-NN completed in {brute_force_time:.2f} seconds.")

    # LSH-based K-NN
    print("Performing LSH-based K-NN...")
    threshold=0.4
    for num_perm in [16,32,64]:

        start_time = time()
        lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
        train_minhashes = [create_minhash(list_of_train_shingle[traindex], num_perm) for traindex in range(len(list_of_train_shingle))]
        
        for i, minhash in enumerate(train_minhashes):
            lsh.insert(str(i), minhash)
        
        build_time = time() - start_time
        start_time = time()
        lsh_results = []
        
        for teindex in range(len(list_of_test_shingle)):
            test_minhash = create_minhash(list_of_test_shingle[teindex], num_perm)
            candidates = lsh.query(test_minhash)
            lsh_results.append([int(cand) for cand in candidates])
        
        query_time = time() - start_time
        
        matched_fractions = []
        for brute_neighbors, lsh_neighbors in zip(nearest_neighbors, lsh_results):
            brute_set=set(brute_neighbors)
            lsh_set=set(lsh_neighbors)
            matched_count = len(brute_set.intersection(lsh_set))
            matched_fraction = matched_count / len(brute_set) if brute_set else 0
            matched_fractions.append(matched_fraction)
        
        average_fraction = np.mean(matched_fractions) if matched_fractions else 0
        
        print(f"LSH Results (num_perm={num_perm},threshold={threshold}):")
        print(f"  Build Time: {build_time:.2f} seconds")
        print(f"  Query Time: {query_time:.2f} seconds")
        print(f"  Fraction Matched: {average_fraction:.2f}")
if __name__ == "__main__":
    nearest_neighbor_search()


Loading the training and the test datasets...
Splitting the training dataset to a smaller one (fraction:10.0%) ...
Splitting the testing dataset to a smaller one (fraction:10.0%) ...
Performing preprocessing and shingling of the data...
Shingling the training set... (k=3)
Shingling the test set... (k=3)
Performing Brute-Force K-NN...
Brute-Force K-NN completed in 5544.29 seconds.
Performing LSH-based K-NN...
LSH Results (num_perm=16,threshold=0.4):
  Build Time: 91.79 seconds
  Query Time: 44.16 seconds
  Fraction Matched: 0.43
LSH Results (num_perm=32,threshold=0.4):
  Build Time: 94.69 seconds
  Query Time: 43.36 seconds
  Fraction Matched: 0.33
LSH Results (num_perm=64,threshold=0.4):
  Build Time: 99.11 seconds
  Query Time: 43.26 seconds
  Fraction Matched: 0.17
